In [2]:
import pandas as pd
import numpy as np
import os, os.path
import pathlib
import json
import re
from functools import partial
from datetime import date
from statistics import mean
import collections
from sklearn.feature_extraction.text import CountVectorizer

In [3]:
# zbiorcza ramka do głownej analizy
merged_message = pd.DataFrame(
    {"file" : [],
    "participants" : [],
     "mean_my_res_time" : [],
     "mean_other_res_time" :[],
    "my_messages" : [],
     "all_messages" : []
    },
    )

merged_common_words = pd.DataFrame(
    {"word" : [],
    "my_count" : [],
    "other_count" : [],
     "file" : [],
    },
    )

merged_time = pd.DataFrame(
    {"date" : [],
     "file" : []
    },
    )


In [4]:
# aby emotki działały
fix_mojibake_escapes = partial(
        re.compile(rb'\\u00([\da-f]{2})').sub,
        lambda m: bytes.fromhex(m[1].decode()),
    )

In [5]:
#stopwords = set(line.strip() for line in open('stopwords.txt', 'utf8'))
stopwords = set(line.strip() for line in open('stopwords.txt', encoding='utf8'))

In [6]:
        me = "Łukasz Grabarski"

In [7]:
def get_wordcount(content):
    content = ' '.join(map(str, content))
    wordcount = {}
    for word in content.lower().split():
        word = word.replace(".","")
        word = word.replace(",","")
        word = word.replace("nan","")
        word = word.replace("-"," ")
        word = word.replace(":","")
        word = word.replace("\"","")
        word = word.replace("!","")
        word = word.replace("*","")
        if word not in stopwords:
            if word not in wordcount:
                wordcount[word] = 1
            else:
                wordcount[word] += 1
                
    word_counter = collections.Counter(wordcount)
    
    if word_counter.get("") != None:
        word_counter.pop("")

    word_counter = (pd.DataFrame(list(word_counter.most_common())))
    
    return word_counter
    

In [8]:
    #idziemy po wzystkich jsonach w folderze
for file in pathlib.Path('inbox').glob('**/*.json'):
        rozmowa = os.path.basename(file.parent).split('_')[0]    

        #importujemy plik z działającymi polskimi znakami
        with open(file, 'rb') as binary_data:
            repaired = fix_mojibake_escapes(binary_data.read())
        data = json.loads(repaired)

        #normalna eksploracja danych
        df_participants = pd.DataFrame(data['participants'])
        df_messages = pd.DataFrame(data['messages'])
        df_messages['date'] = pd.to_datetime(df_messages.timestamp_ms, unit = 'ms')

        participants = df_participants['name'].size
        mean_other_res_time = -1
        mean_my_res_time = -1
        my_messages = len(df_messages[df_messages.sender_name == me])
        all_messages = len(df_messages['sender_name'])
        title = data['title']

        #badamy czas odpowiedzi
        if participants == 2:
            senders = list(df_messages['sender_name'])
            times =  list(df_messages['timestamp_ms'])
            times.reverse()
            senders.reverse()
            response_time_my = list()
            response_time_other = list()
            iprev = 0

            for i in range(len(times)):
                if senders[i] != senders[iprev]:
                    if senders[i] == me:
                        response_time_my.append(times[i] - times[iprev])
                    else:
                        response_time_other.append(times[i] - times[iprev])

                iprev = i

            if len(response_time_my) > 2:          
                mean_my_res_time = round(mean(response_time_my))
            if len(response_time_other) > 2:          
                mean_other_res_time = round(mean(response_time_other))

        # najczęstrze słowa
        
        if 'content' not in df_messages:
            continue
        
        #liczymy moje słowa
        my_content = list(df_messages.query('sender_name == @me')['content'])
        my_word_counter = get_wordcount(my_content)
        my_word_counter.rename(columns = {0 : 'word', 1 : 'my_count'}, inplace = True)
        
        if len(my_word_counter.index) == 0:
            continue
        
        #liczymy słowa innych
        other_content = list(df_messages.query('sender_name != @me')['content'])
        other_word_counter = get_wordcount(other_content)
        other_word_counter.rename(columns = {0 : 'word', 1 : 'other_count'}, inplace = True)
        
        if len(other_word_counter.index) == 0:
            continue
        
        #mergujemy
        word_counter = pd.merge(my_word_counter, other_word_counter,
                               how = 'inner',
                               on = 'word')
        word_counter.fillna(0)
        word_counter['file'] = title

        #czas wysłania wiadomości
        
        messages_time = pd.DataFrame(df_messages.query('sender_name == @me')['date'])
        messages_time['file'] = title


        #mergujemy z główną ramką
        df_to_merge = pd.DataFrame({"file" : title,
                                    "participants" : participants,
                                    "mean_my_res_time" : mean_my_res_time,
                                    "mean_other_res_time" : mean_other_res_time,
                                    "my_messages" : my_messages,
                                    "all_messages" : all_messages},
                                  index=[1])

        print(title)
        merged_message = pd.concat([merged_message, df_to_merge])
        merged_common_words = pd.concat([merged_common_words, word_counter])
        merged_time = pd.concat([merged_time, messages_time])

18 Łukasza🎉
3B Chłopacy
3B eWychowawcza
3D
3D
4min4life <3
9 lutego ✨
Adam Ciborek
Adam Dęby
Adam Kąkolewski
Adam Krystek
Adam Przemysław Chojecki
Adrian Cieśla
Adrian Gołębiowski
Adrian Kaźmierczuk
Adrian Zaręba
Aga Mańke
Aga Mańke
Aga Mańke
Aga Mańke
Aga Mańke
Aga Mańke
Aga Mańke
Agata Andrzejczak
Agata Boruta
Agata Chałupniczak
Agata Gerl
Agata i Zosia
Agata Kaczmarek
Agata Makarewicz
Agata, Szy, Zosia i 3 innych użytkowników
Agnieszka Balcerowiak
Agnieszka Dziadek
Agnieszka Grabarska
Agnieszka Kaczmarek
AJR
AKCJA: WARSZAFKA
Alan Kirpa
Ala Röhl
Alek Malinowski
Aleksander Mieszała
Aleksandra Drozd
Aleksandra Dworzak
Aleksandra Grabarska
Aleksandra i Magdalena
Aleksandra Konieczka
Aleksandra Kulczycka
Aleksandra Ortyl
Aleksandra Pela
Aleksandra Piotrowicz
Aleksandra Spychała
Aleksandra Sroka
Aleks Kapich
Alicja Charuza
Alicja Grześków
Alicja Nowicka
Amelia Janc
Amelia Łakoma
Amelia Spirra
Andrii Voznesenskyi
Andrzej Łuczak
Andrzej Mostaller
Ania, Kacper i Aga
Ania Kuska
Ania Linka
Ani

Rozmowni ludzie💕
Roztańczeni💃🏻🕺🏻❤
RP - płakanie
Rrrr
Samorząd Studentów SGH
Samouczki fan club
Sandra Kowalska
Schronisko Bukowina - Restauracja i Sklep
Sebastian Tritt
Socjal 2k22
Spam WRS MiNI 2021
Spam WRS MiNI 2022
🇺🇸Spiskowe Teorie Majewskiej🇺🇸
Spotted: Budzyń i okolice
Spotted: Chodzież
Spotted: Złotów
SSH SSH
Stanisław Białka
Stanisław Wiśniewski
Starostowie
Start-up Society Poznań
Studniówka
Światła, Kamera, Akcja!
Szefostwo Na Wrzesień
Szkolenie 4 Jeźdźców Apokalipsy
Szkolenie Centralne SSPW 👾🤖
Szkolna Rada Uczniowska w VIII LO w Poznaniu
SzRU 2018/19
SzRU 2019/20
SzRU 2020/21 [OFFTOP]
SzRU 2021/22
Sztab Wyborczy❤
Szy Cy
Szymon Abramczyk
Szymon Kin
Szymon Tuz
TEDxMickiewiczHighSchool
Teksiciarze
Tomasz i Zosia
Tomasz Poprawski
Tomasz Witkowski
Tomek Garstka
Tomek i Aga
Tomek, Maja, Nikodem i 2 innych użytkowników
Tomek, Mikołaj, Krzysztof i Michał
Tomek, Mikołaj, Lena i 2 innych użytkowników
To Nie Ja
tooo late
Trójki Klasowe 2021/22
Trójki Klasowe 8LO
Truskaweczki 
TVPW
Twd
T

#df['date'] = pd.to_datetime(df.timestamp_ms, unit='ms') <- linijka która zamienia ten dziwny czas timestamp na normlaną datę
merged_message.to_csv(me + '_'+ 'merged_message.csv')
merged_common_words.to_csv(me + '_'+ 'merged_common_words.csv')
merged_time.to_csv(me + '_'+ 'merged_time.csv')


In [9]:
merged_common_words

,word,my_count,other_count,file
0,ankieta,3.0,20.0,18 Łukasza🎉
1,dostępna,3.0,20.0,18 Łukasza🎉
2,rano,1.0,1.0,18 Łukasza🎉
3,muzyka,1.0,1.0,18 Łukasza🎉
4,xd,1.0,3.0,18 Łukasza🎉
...,...,...,...,...
15,rzeczy,1.0,1.0,Zuza Rybacka
16,wiesz,1.0,2.0,Zuza Rybacka
17,prostu,1.0,1.0,Zuza Rybacka
18,raz,1.0,2.0,Zuza Rybacka


In [10]:
merged_message

,file,participants,mean_my_res_time,mean_other_res_time,my_messages,all_messages
1,18 Łukasza🎉,15.0,-1.0,-1.000000e+00,54.0,123.0
1,3B Chłopacy,17.0,-1.0,-1.000000e+00,269.0,7866.0
1,3B eWychowawcza,35.0,-1.0,-1.000000e+00,40.0,2023.0
1,3D,15.0,-1.0,-1.000000e+00,832.0,10000.0
1,3D,15.0,-1.0,-1.000000e+00,183.0,2073.0
...,...,...,...,...,...,...
1,Zuzanna Piosik,2.0,61227.0,1.074140e+05,10.0,19.0
1,Zuzanna Piróg,2.0,167826924.0,7.367044e+07,161.0,410.0
1,Zuzanna Pszczółka,2.0,2207669.0,1.943588e+08,5.0,12.0
1,Zuzanna Schiller,2.0,627177704.0,9.671451e+09,8.0,15.0


In [11]:
merged_time

,date,file
1,2020-02-04 17:36:45.008000,18 Łukasza🎉
3,2020-02-04 17:32:56.077000,18 Łukasza🎉
4,2020-02-04 17:32:33.670000,18 Łukasza🎉
5,2020-02-04 17:31:23.387000,18 Łukasza🎉
6,2020-02-04 17:30:42.959000,18 Łukasza🎉
...,...,...
125,2020-11-19 13:58:17.789000,Zuza Rybacka
126,2020-11-19 13:58:15.844000,Zuza Rybacka
129,2020-11-19 13:57:39.465000,Zuza Rybacka
130,2020-11-19 13:56:05.094000,Zuza Rybacka
